# 第三章：输入数据格式 — CP2K 输出文件解析

本章深入理解两个输入文件的格式：
1. `md.inp` — CP2K 输入文件（含晶胞参数）
2. `md-pos-1.xyz` — 原子轨迹文件（274 帧）

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
sys.path.insert(0, str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / "data_example" / "potential"
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

import numpy as np
import matplotlib.pyplot as plt

print("路径设置完成")

---

## 3.1 `md.inp` — CP2K 输入文件

这是 CP2K 程序的输入文件，包含 MD 模拟的所有设置参数。  
本项目只需要从中提取**晶胞参数**（模拟盒子的尺寸）。

### 文件内容预览

In [ ]:
md_inp_path = DATA_DIR / "md.inp"

# 读取并显示文件内容
内容 = md_inp_path.read_text(encoding='utf-8')
print(内容)

### 关键行：`ABC [angstrom] a b c`

在文件中找到 `ABC [angstrom]` 这一行，它定义了**正交晶胞**的三个边长：
- `a` — x 方向长度（Å）
- `b` — y 方向长度（Å）  
- `c` — z 方向长度（Å）**（垂直于金属板的方向，即分析方向）**

正交晶胞示意图：
```
      c (z方向，垂直金属板)
      ↑
      │   ____b____
      │  /        /|
      │ /________/ |
      │ |        | |
      │ |        | /
      │ |________|/ 
      └──────────────→ a (x方向)
```

In [ ]:
import re

# 用正则表达式找到 ABC 行
# re.search 在文本中搜索匹配的模式
匹配 = re.search(
    r'^\s*ABC\s+\[angstrom\]\s+([0-9.eE+-]+)\s+([0-9.eE+-]+)\s+([0-9.eE+-]+)',
    内容,
    re.MULTILINE  # 多行模式：^ 匹配每行的开头
)

if 匹配:
    # 用项目内置的解析函数
    from src.structure.Analysis.WaterAnalysis._common import _parse_abc_from_md_inp
    a_A, b_A, c_A = _parse_abc_from_md_inp(md_inp_path)
    
    print("✅ 成功解析晶胞参数：")
    print(f"   a = {a_A:.6f} Å  （x 方向）")
    print(f"   b = {b_A:.6f} Å  （y 方向）")
    print(f"   c = {c_A:.6f} Å  （z 方向，分析轴）")
    print()
    print(f"   晶胞体积 = {a_A * b_A * c_A:.2f} Å³")
    print(f"   xy 截面积 = {a_A * b_A:.2f} Å²")
else:
    print("❌ 未找到 ABC 行")

### 文件中的其他信息（本项目不使用，但了解有帮助）

| 关键字 | 含义 |
|--------|------|
| `RUN_TYPE MD` | 运行分子动力学模拟 |
| `ENSEMBLE NVT` | 等温等体积系综（温度恒定） |
| `STEPS 20000` | 共 20000 步，每5步输出一次轨迹 → 4000帧（实际274帧是子集） |
| `TIMESTEP 1.0` | 时间步长 1.0 fs |
| `TEMPERATURE 300.15` | 温度约 300K（室温）|

---

## 3.2 `md-pos-1.xyz` — 轨迹文件

### XYZ 文件格式

XYZ 格式是最简单的原子结构格式，每帧（快照）由三部分组成：

```
行1: 原子总数
行2: 注释行（时间步、能量等，可以是任意文本）
行3起: 每行一个原子 → 元素符号  x坐标  y坐标  z坐标
...
（重复，下一帧接着写）
```

### 查看文件的前几行

In [ ]:
xyz_path = DATA_DIR / "md-pos-1.xyz"

# 读取前 15 行
with open(xyz_path, 'r') as f:
    前15行 = [f.readline() for _ in range(15)]

print("=== md-pos-1.xyz 前 15 行 ===")
for i, 行 in enumerate(前15行, start=1):
    print(f"第{i:2d}行: {行}", end='')

print("\n=== 格式解释 ===")
print(f"第1行 '{前15行[0].strip()}' → 本帧共有这么多原子")
print(f"第2行 '{前15行[1].strip()}' → 注释行（帧号 i 和能量 E）")
print(f"第3行 '{前15行[2].strip()}' → 第1个原子：元素 + x + y + z 坐标")

### 用 ASE 读取所有帧

`ase.io.read(..., index=':')` 读取所有帧：
- `index=0`：只读第一帧
- `index=-1`：只读最后一帧
- `index=':5'`：读取第0到第4帧（共5帧）
- `index=':'`：读取所有帧

> ⚠️ 读取全部 274 帧会花几秒钟

In [ ]:
import ase.io

print("正在读取所有帧...")
所有帧 = ase.io.read(str(xyz_path), index=':')

print(f"\n读取完成：")
print(f"  总帧数: {len(所有帧)} 帧")

# 设置晶胞（XYZ 不含晶胞，需要手动设置）
for frame in 所有帧:
    frame.set_cell([a_A, b_A, c_A])
    frame.set_pbc([True, True, True])

第一帧 = 所有帧[0]
print(f"  每帧原子数: {len(第一帧)}")

# 统计元素
import numpy as np
元素 = np.array(第一帧.get_chemical_symbols())
from collections import Counter
元素统计 = Counter(元素)
print(f"  第一帧元素统计:")
for 元素名, 数量 in sorted(元素统计.items(), key=lambda x: -x[1]):
    print(f"    {元素名}: {数量} 个")

### 可视化：原子 z 坐标随帧变化

轨迹就是一系列快照。下图展示在整个 MD 过程中，
**部分代表性原子**的 z 坐标如何随时间变化。

- 金属原子（Cu）：z 坐标几乎不变（金属板固体，热振动小）
- 水原子（O）：z 坐标在金属板两侧随机运动（液态水扩散）

In [ ]:
# 提取各帧中选定原子的 z 坐标
帧数 = len(所有帧)
帧编号 = np.arange(帧数)

# 选几个代表性原子：前几个 Cu 原子和几个 O 原子
第一帧元素 = np.array(第一帧.get_chemical_symbols())
Cu编号列表 = np.where(第一帧元素 == 'Cu')[0][:3]  # 前3个 Cu
O编号列表  = np.where(第一帧元素 == 'O')[0][:3]   # 前3个 O

# 提取这些原子在各帧的 z 坐标
Cu_z_轨迹 = np.array([[所有帧[i].get_positions()[idx, 2] for i in range(帧数)] for idx in Cu编号列表])
O_z_轨迹  = np.array([[所有帧[i].get_positions()[idx, 2] for i in range(帧数)] for idx in O编号列表])

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

# 上图：Cu 原子的 z 轨迹
for j, idx in enumerate(Cu编号列表):
    ax1.plot(帧编号, Cu_z_轨迹[j], linewidth=0.8, label=f'Cu 原子 #{idx}')
ax1.set_ylabel('z 坐标 (Å)', fontsize=11)
ax1.set_title('金属原子（Cu）的 z 坐标随帧变化', fontsize=12)
ax1.legend(fontsize=9)

# 下图：O 原子的 z 轨迹
for j, idx in enumerate(O编号列表):
    ax2.plot(帧编号, O_z_轨迹[j], linewidth=0.8, label=f'O 原子 #{idx}')
ax2.set_xlabel('帧编号', fontsize=11)
ax2.set_ylabel('z 坐标 (Å)', fontsize=11)
ax2.set_title('水分子（O）的 z 坐标随帧变化', fontsize=12)
ax2.legend(fontsize=9)

plt.tight_layout()
plt.show()

print("观察：")
print(f"  Cu 原子 z 坐标标准差: {Cu_z_轨迹.std():.3f} Å  (小 → 金属板基本不动)")
print(f"  O  原子 z 坐标标准差: {O_z_轨迹.std():.3f} Å   (大 → 水分子到处跑)")

---

## 3.3 数据统计总览

In [ ]:
# 计算水分子数（O原子数 = 水分子数，因为每个 O 对应一个水分子）
O数量 = (第一帧元素 == 'O').sum()
H数量 = (第一帧元素 == 'H').sum()
金属数量 = (~np.isin(第一帧元素, ['O', 'H'])).sum()

print("=== 数据集统计 ===")
print(f"  轨迹帧数: {帧数}")
print(f"  每帧原子数: {len(第一帧)}")
print(f"    金属原子 (Cu+Ag): {金属数量}")
print(f"    水 O 原子: {O数量} (= {O数量} 个水分子)")
print(f"    水 H 原子: {H数量} (= {O数量}×2 = {O数量*2})")
print(f"  晶胞尺寸: a={a_A:.3f}, b={b_A:.3f}, c={c_A:.3f} Å")
print(f"  体相水密度参考: ~1.0 g/cm³")

In [ ]:
# 柱状图：第一帧中各元素的原子数量
from collections import Counter

元素计数 = Counter(第一帧元素)
元素名称 = list(元素计数.keys())
元素数量 = [元素计数[e] for e in 元素名称]

# 按原子数量排序（从多到少）
排序索引 = np.argsort(元素数量)[::-1]
元素名称排序 = [元素名称[i] for i in 排序索引]
元素数量排序 = [元素数量[i] for i in 排序索引]

# 颜色：金属用橙色，O用红色，H用蓝色
颜色映射 = {'O': 'tab:red', 'H': 'tab:blue', 'Cu': 'tab:orange', 'Ag': 'silver'}
颜色列表 = [颜色映射.get(e, 'tab:gray') for e in 元素名称排序]

fig, ax = plt.subplots(figsize=(7, 4))
bars = ax.bar(元素名称排序, 元素数量排序, color=颜色列表, edgecolor='black', linewidth=0.5)

# 在柱顶标注数值
for bar, 数量 in zip(bars, 元素数量排序):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            str(数量), ha='center', va='bottom', fontsize=10)

ax.set_xlabel('元素', fontsize=12)
ax.set_ylabel('原子数量', fontsize=12)
ax.set_title('第一帧中各元素的原子数量', fontsize=13)

plt.tight_layout()
plt.show()

---

## 本章小结

| 文件 | 作用 | 关键信息 |
|------|------|----------|
| `md.inp` | CP2K 输入文件 | 从 `ABC [angstrom] a b c` 获取晶胞参数 |
| `md-pos-1.xyz` | 原子轨迹 | 274 帧，每帧 274 个原子（Cu+Ag+O+H） |

**数据集概述**：
- 274 帧 MD 轨迹，每帧含金属原子和水分子
- 金属板（Cu/Ag）基本固定，水分子在两侧液态运动
- 晶胞为正交型（直角），c 轴为分析方向

**下一章**（`04_layer1_metal_detection.ipynb`）将分析如何从原子坐标中自动识别金属层和界面层。